In [1]:
import os
import gc
import re
import sys
import time
import json
import math
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

In [2]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_new = pd.read_csv('../tmp/train/car_autohome_new.csv')
car_autohome_new = car_autohome_new.drop(['volume', 'control'], axis=1)
car_autohome_new['']
global_model_mean = pd.read_csv('../tmp/train/global_model_mean.csv')

In [5]:
car_autohome_all.head()

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,online_year,volume,control
0,454,大众,1,宝来/宝来经典,15,2004款 1.6 手动基本,一汽-大众,13.50,紧凑型车,汽油,2004,1.6,手动
1,456,大众,1,宝来/宝来经典,15,2004款 1.6 自动基本,一汽-大众,14.70,紧凑型车,汽油,2004,1.6,自动
2,466,大众,1,宝来/宝来经典,15,2004款 1.6 自动天窗,一汽-大众,15.88,紧凑型车,汽油,2004,1.6,自动
3,516,大众,1,宝来/宝来经典,15,2004款 1.8 手动尊贵,一汽-大众,19.39,紧凑型车,汽油,2004,1.8,手动
4,518,大众,1,宝来/宝来经典,15,2004款 1.8 自动尊贵,一汽-大众,20.69,紧凑型车,汽油,2004,1.8,自动


In [4]:
car_autohome_new.head()

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,online_year,volume,control
0,37063,大众,1,捷达,16,2019款 梦想版 1.4L 手动时尚型,一汽-大众,8.11,紧凑型车,汽油,2019,1.4,手动
1,37064,大众,1,捷达,16,2019款 梦想版 1.4L 手动舒适型,一汽-大众,9.31,紧凑型车,汽油,2019,1.4,手动
2,37065,大众,1,捷达,16,2019款 梦想版 1.5L 手动时尚型,一汽-大众,8.68,紧凑型车,汽油,2019,1.5,手动
3,37066,大众,1,捷达,16,2019款 梦想版 1.5L 自动时尚型,一汽-大众,9.68,紧凑型车,汽油,2019,1.5,自动
4,37067,大众,1,捷达,16,2019款 梦想版 1.5L 手动舒适型,一汽-大众,9.88,紧凑型车,汽油,2019,1.5,手动
